In [ ]:
!git clone https://github.com/VITA-Group/Peek-a-Boo.git

Cloning into 'Peek-a-Boo'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 63 (delta 21), reused 63 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 67.52 KiB | 13.50 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
%cd Peek-a-Boo

/content/Peek-a-Boo


In [ ]:
!pip install tqdm
import torch

In [ ]:
from torch.hub import load_state_dict_from_url

In [ ]:
!python main_cifar.py --use-cuda 0 \
    --arch PsgResNet18 --init-method standard \
    --optimizer BOP --ar 1e-3 --tau 1e-6 \
    --ar-decay-freq 45 --ar-decay-ratio 0.15 --epochs 2 \
    --pruner SynFlow --prune-epoch 0 \
    --prune-ratio 3e-1 --prune-iters 100 \
    --msb-bits 8 --msb-bits-weight 8 --msb-bits-grad 16 \
    --psg-threshold 1e-7 --psg-no-take-sign --psg-sparsify \
    --exp-name cifar10_resnet18_pab-psg

[2024-11-20 07:23:45,666] [main] ==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[2024-11-20 07:23:47,251] [main] ==> Building model..
[2024-11-20 07:23:47,755] [main] ==> An initialization checkpoint exists. Resuming from the initialization..
/content/Peek-a-Boo/main_cifar.py:298: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execut

In [ ]:
import torch

In [ ]:
state_dict=torch.load("/content/ckpt_best (1).pth",map_location=torch.device('cpu'))

<ipython-input-4-86881cec8bf9>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load("/content/ckpt_best (1).pth",map_location=torch.device('cpu'))


In [ ]:
state_dict["net"]

OrderedDict([('conv1.weight',
              tensor([[[[-0.1742, -0.1823, -0.1421],
                        [-0.1363, -0.0250, -0.0789],
                        [-0.1352, -0.0030, -0.0374]],
              
                       [[-0.0293, -0.0636,  0.1640],
                        [-0.1547, -0.0660,  0.0882],
                        [-0.0414, -0.1314, -0.1902]],
              
                       [[-0.1650,  0.1057,  0.1175],
                        [-0.0475,  0.1559,  0.1863],
                        [-0.0478, -0.1499,  0.0076]]],
              
              
                      [[[ 0.0605, -0.0206, -0.0860],
                        [ 0.0520,  0.1083, -0.1574],
                        [ 0.1131,  0.1802, -0.0487]],
              
                       [[-0.1222, -0.1722, -0.1544],
                        [-0.1031, -0.0098, -0.1348],
                        [-0.0380, -0.0845, -0.1675]],
              
                       [[ 0.1272,  0.1558,  0.0730],
                        [ 

In [ ]:
state_dict=state_dict["net"]

In [ ]:


for key, value in state_dict.items():
    if 'weight' in key and 'mask' not in key:  # Find weight layers
        mask_key = key.replace('weight', 'weight_mask')  # Assumes corresponding mask keys
        if mask_key in state_dict:  # Ensure mask exists
            state_dict[key] *= state_dict[mask_key]  # Update weight with mask


In [ ]:
state_dict

OrderedDict([('conv1.weight',
              tensor([[[[-0.1742, -0.1823, -0.1421],
                        [-0.1363, -0.0250, -0.0789],
                        [-0.1352, -0.0030, -0.0374]],
              
                       [[-0.0293, -0.0636,  0.1640],
                        [-0.1547, -0.0660,  0.0882],
                        [-0.0414, -0.1314, -0.1902]],
              
                       [[-0.1650,  0.1057,  0.1175],
                        [-0.0475,  0.1559,  0.1863],
                        [-0.0478, -0.1499,  0.0076]]],
              
              
                      [[[ 0.0605, -0.0206, -0.0860],
                        [ 0.0520,  0.1083, -0.1574],
                        [ 0.1131,  0.1802, -0.0487]],
              
                       [[-0.1222, -0.1722, -0.1544],
                        [-0.1031, -0.0098, -0.1348],
                        [-0.0380, -0.0845, -0.1675]],
              
                       [[ 0.1272,  0.1558,  0.0730],
                        [ 

In [ ]:
torch.save(state_dict, "disguised(1)_masked.pth")